<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosI/blob/master/notebooks/3e_lsh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda del vecino más cercano aproximado mediante funciones _hash_ sensibles a la localidad
En esta libreta se realiza un buscador del vecino más cercano aproximado usando funciones _hash_ sensibles a la localidad (LSH). Especificamente, se define la familia LSH basada  en distribuciones $p$-estables para distancias $\ell_1$ y $\ell_2$ y otra familia para la distancia angular.

In [1]:
from abc import ABC, abstractmethod 

from os import listdir
from os.path import isfile, join
import struct

import os 
import time

import numpy as np

N_TOP = 1

## Conjunto de datos
Para evaluar el buscador vamos usar el conjunto de vectores SIFT [ANN_SIFT10K](http://corpus-texmex.irisa.fr/) del grupo TEXMEX, el cual descargamos y extraemos.

In [2]:
!wget -q ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz
!tar xvzf siftsmall.tar.gz

siftsmall/
siftsmall/siftsmall_base.fvecs
siftsmall/siftsmall_groundtruth.ivecs
siftsmall/siftsmall_learn.fvecs
siftsmall/siftsmall_query.fvecs


Definimos una función para leer los vectores de un archivo `.fvecs`.

In [3]:
import struct
import os 

def lee_fvecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d))
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('f' * d, f.read(d * 4))
  
  return vecs 

Leemos el conjunto de vectores base y consulta.

In [4]:
base = lee_fvecs('siftsmall/siftsmall_base.fvecs')
consultas = lee_fvecs('siftsmall/siftsmall_query.fvecs')

print('Base: {0} Consultas: {1}'.format(base.shape, consultas.shape))

Base: (10000, 128) Consultas: (100, 128)


Definimos una función para leer los vectores más cercanos reales (_groundtruth_) de un archivo `.ivecs`

In [5]:
def lee_ivecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d), dtype=int)
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('i' * d, f.read(d * 4))
  
  return vecs 

Leemos estos vectores.

In [6]:
gt = lee_ivecs('siftsmall/siftsmall_groundtruth.ivecs')
print('Groundtruth: {0}'.format(gt.shape))

Groundtruth: (100, 100)


In [7]:
gt[0]

array([2176, 3752,  882, 4009, 2837,  190, 3615,  816, 1045, 1884,  224,
       3013,  292, 1272, 5307, 4938, 1295,  492, 9211, 3625, 1254, 1292,
       1625, 3553, 1156,  146,  107, 5231, 1995, 9541, 3543, 9758, 9806,
       1064, 9701, 4064, 2456, 2763, 3237, 1317, 3530,  641, 1710, 8887,
       4263, 1756,  598,  370, 2776,  121, 4058, 7245, 1895,  124, 8731,
        696, 4320, 4527, 4050, 2648, 1682, 2154, 1689, 2436, 2005, 3210,
       4002, 2774,  924, 6630, 3449, 9814, 3515, 5375,  287, 1038, 4096,
       4094,  942, 4321,  123, 3814,   97, 4293,  420, 9734, 1916, 2791,
        149, 6139, 9576, 6837, 2952, 3138, 2890, 3066, 2852,  348, 3043,
       3687])

In [8]:
base[2176]

array([  0.,   1.,   3.,   0.,   5.,  86.,  60.,   0.,   0.,   0.,   3.,
         0.,   2., 103., 118.,   9.,   0.,   0.,   0.,   3.,   8.,  42.,
       118.,  54.,   2.,   0.,   0.,   1.,   5.,  34.,  26.,  30.,   0.,
         0.,   0.,  24.,  83., 105.,  41.,   0.,   1.,   0.,   0.,  28.,
        85., 105., 100.,  15.,  60.,   8.,   1.,   2.,   5.,  17.,  88.,
       118.,  39.,   2.,   6.,   8.,   7.,  22.,  19.,  81.,  19.,   7.,
         5.,  37.,  46.,  22.,  28.,  15.,  17.,  16.,  31., 118.,  61.,
         1.,   1.,   2., 118.,  88.,  22.,  67.,   6.,   1.,   1.,   8.,
        68.,  34.,  25.,  73.,  14.,   2.,   3.,  25.,  67.,  10.,   1.,
        24.,  41.,   1.,   6.,  40.,  67.,  23.,  26., 118.,  45.,   4.,
         1.,  25.,  56.,  69.,  71.,  70.,  15.,   1.,   1.,   1.,  36.,
        18.,  10.,  33.,  41.,  13.,   5.,   1.])

In [9]:
consultas[0]

array([  1.,   3.,  11., 110.,  62.,  22.,   4.,   0.,  43.,  21.,  22.,
        18.,   6.,  28.,  64.,   9.,  11.,   1.,   0.,   0.,   1.,  40.,
       101.,  21.,  20.,   2.,   4.,   2.,   2.,   9.,  18.,  35.,   1.,
         1.,   7.,  25., 108., 116.,  63.,   2.,   0.,   0.,  11.,  74.,
        40., 101., 116.,   3.,  33.,   1.,   1.,  11.,  14.,  18., 116.,
       116.,  68.,  12.,   5.,   4.,   2.,   2.,   9., 102.,  17.,   3.,
        10.,  18.,   8.,  15.,  67.,  63.,  15.,   0.,  14., 116.,  80.,
         0.,   2.,  22.,  96.,  37.,  28.,  88.,  43.,   1.,   4.,  18.,
       116.,  51.,   5.,  11.,  32.,  14.,   8.,  23.,  44.,  17.,  12.,
         9.,   0.,   0.,  19.,  37.,  85.,  18.,  16., 104.,  22.,   6.,
         2.,  26.,  12.,  58.,  67.,  82.,  25.,  12.,   2.,   2.,  25.,
        18.,   8.,   2.,  19.,  42.,  48.,  11.])

## Distancias $\ell_1$ y $\ell_2$.
Definimos nuestra clase de tabla _hash_ con una familia de funciones basada en distribuciones $s$-estables. En esta familia se elige aleatoriamente una proyección de $\mathbb{R}^d$ sobre una línea, se desplaza por $b$ y se corta en segmentos de tamaño $w$, esto es,
        $$
        h_{\mathbf{a},b} = \left\lfloor  \frac{\mathbf{a} \cdot \mathbf{x} + b}{w} \right\rfloor
        $$
donde $b \in [0, w)$

 * Si $\mathbf{a}$ se muestrea de una distribución normal se obtiene una familia LSH para distancia $\ell_2$.\newline
 * Si $\mathbf{a}$ se muestrea de una distribución de Cauchy se obtiene una familia LSH para distancia $\ell_1$

In [10]:
class TablaLSH(ABC):
  def __init__(self, n_cubetas, t_tupla, dim):
    self.n_cubetas = n_cubetas
    self.t_tupla = t_tupla
    self.dim = dim
    self.tabla = [[] for i in range(n_cubetas)]
    
    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291

  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo
  
  def tuplehash(self, x):
    hv = np.sum(self.a * x, dtype=np.ulonglong)
    idx = np.sum(self.b * x, dtype=np.ulonglong)
    return hv, idx

  def insertar(self, x, ident):
    hv, v2 = self.lshfun(x)

    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(hv)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == hv:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    hv, v2 = self.lshfun(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == hv:
        return self.tabla[cubeta][1]
        
    return []
  
  @abstractmethod
  def lshfun(self, x):
    pass

Creamos una clase para generar, construir y buscar vectores en múltiples tablas _hash_.

In [11]:
class EstructuraLSH:
  def __init__(self, FamLSH, n_tablas, t_tabla, t_tupla, dim, **kwargs):
    self.n_tablas = n_tablas
    self.tablas = [FamLSH(t_tabla, t_tupla, dim, **kwargs) for _ in range(n_tablas)]

  def construir(self, base):
    for i,x in enumerate(base):
      for t in range(self.n_tablas):
        self.tablas[t].insertar(x, i)

  def buscar(self, consultas):
    prom_docrec = 0
    vecs = []
    for i,q in enumerate(consultas):
      dc_lp = []
      for t in range(len(self.tablas)):
          dc_lp.extend(self.tablas[t].buscar(q))
      prom_docrec += len(set(dc_lp))
      vecs.append(set(dc_lp))

    return vecs, prom_docrec

Definimos la subclase de tabla _hash_ para la familia LSH de distribuciones $p$ estables.

In [12]:
class TablaLSHLpDist(TablaLSH):
  def __init__(self, n_cubetas, t_tupla, dim, **kwargs):
    super().__init__(n_cubetas, t_tupla, dim)
    self.w = kwargs['width']

    if kwargs['norma'] == 'l2':
      self.Amat = np.random.standard_normal((t_tupla, dim)) 
    elif kwargs['norma'] == 'l1':
      self.Amat = np.random.standard_cauchy((t_tupla, dim))

    self.bvec = np.random.uniform(low=0, high=self.w, size=t_tupla)

  def lshfun(self, x):
    prod = np.floor((self.Amat @ x.T[:, np.newaxis] + self.bvec[:, np.newaxis]) / self.w).astype(np.uint32)
    return self.tuplehash(prod)

Creamos funciones para calcular la distancia euclidiana de un vector consulta con un conjunto de vectores y ordenarlos por su distancia.

In [13]:
def distancia_euclidiana(x, y):   
  return np.sqrt(np.sum((x - y)**2))

def fuerza_bruta(ds, qs, fd):
  medidas = np.zeros(ds.shape[0])
  for i,x in enumerate(ds):
    medidas[i] = fd(qs, x)

  return np.sort(medidas), np.argsort(medidas)

def ordena_recuperados(base, consultas, vecs, fd):
  dists = []
  orden = []
  for i,q in enumerate(consultas):
    ld = list(vecs[i])
    if ld:
      m,o = fuerza_bruta(base[ld], q, fd)
      dists.append(m)
      orden.append([ld[e] for e in o])
    else:
      dists.append([])
      orden.append([])
  
  return orden, dists

Definimos una función para buscar los vecinos más cercanos de un conjunto de vectores de consulta en un conjunto de vectores base almacenados en las tablas _hash_.

In [14]:
def busqueda_lsh(lsh, base, consultas, fd):
  lsh.construir(base)
  start = time.time()
  vecs, prom_docrec = lsh.buscar(consultas)
  orden, dists = ordena_recuperados(base, consultas, vecs, fd)
  end = time.time()

  return orden, dists, prom_docrec, end

Instanciamos la estructura LSH y realizamos la búsqueda.

In [15]:
lplsh = EstructuraLSH(TablaLSHLpDist, 50, 2**14, 20, 128, width = 60, norma = 'l2')
orden, dists, prom_docrec, tiempo = busqueda_lsh(lplsh, base, consultas, distancia_euclidiana)

Comparamos los vecinos más cercanos encontrados por LSH y los reales.

In [16]:
def promedio_correctos(orden, gt, n_top=1):
  vmc_lsh = [o[0] if o else -1 for o in orden]
  vmc_real = [g[:n_top] for g in gt]
  correcto = [vmc_lsh[i] in vmc_real[i] for i in range(len(vmc_lsh))]
  return np.mean(correcto)

Desplegamos los resultados:

In [17]:
print(f'Número de consultas = {len(consultas)}')
print(f'Promedio de documentos recuperados por consulta = {prom_docrec / len(consultas)}')
print(f'Número de consultas por segundo = {len(consultas) / tiempo}')
print(f'Promedio encontrados = {promedio_correctos(orden, gt, n_top=N_TOP)}')

Número de consultas = 100
Promedio de documentos recuperados por consulta = 1048.21
Número de consultas por segundo = 6.065862430487204e-08
Promedio encontrados = 0.36


## Distancia angular
Definimos una clase de tabla LSH para distancia angular $ 1 - \theta(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})$  basada en la siguiente familia 
$$
h_\mathbf{v}(\mathbf{x}^{(i)}) = signo(\mathbf{v} \cdot \mathbf{x}^{(i)})
$$

donde $\mathbf{v} \in \mathbb{R}^d$ es un vector aleatorio de tamaño unitario y

 $$
\theta(\mathbf{x}^{(i)}, \mathbf{x}^{(j)}) = \arccos{\left(\frac{\mathbf{x}^{(i)} \cdot \mathbf{x}^{(j)}}{\lVert \mathbf{x}^{(i)}\rVert \cdot \lVert {\mathbf{x}^{(j)}}\rVert}\right)}
$$

La probabilidad de que cualquier par de vectores $(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})$ tenga un valor idéntico para esta familia es
$$
Pr[h_\mathbf{v}(\mathbf{x}^{(i)}) = h_\mathbf{v}(\mathbf{x}^{(j)}] = 1 - \frac{\theta(\mathbf{x}^{(i)}, \mathbf{x}^{(j)})}{\pi}
$$


In [18]:
class TablaCos(TablaLSH):
  def __init__(self, n_cubetas, t_tupla, dim, **kwargs):
    super().__init__(n_cubetas, t_tupla, dim)
    self.Amat = np.random.standard_normal((t_tupla, dim))

  def lshfun(self, x):
    sign = np.heaviside(self.Amat @ x.T, 1).astype(int)
    return self.tuplehash(sign)


Instanciamos ls estructura LSH con la familia para la distancia angular.

In [19]:
coslsh = EstructuraLSH(TablaCos, 20, 2**14, 20, 128)
orden, dists, prom_docrec, tiempo = busqueda_lsh(coslsh, base, consultas, distancia_euclidiana)

Comparamos los vecinos más cercanos encontrados por LSH y los reales.

In [20]:
print(f'Número de consultas = {len(consultas)}')
print(f'Promedio de documentos recuperados por consulta = {prom_docrec / len(consultas)}')
print(f'Número de consultas por segundo = {len(consultas) / tiempo}')
print(f'Promedio encontrados = {promedio_correctos(orden, gt, n_top=N_TOP)}')

Número de consultas = 100
Promedio de documentos recuperados por consulta = 203.68
Número de consultas por segundo = 6.065862391407264e-08
Promedio encontrados = 0.68
